In [ ]:
# Class for accessing financial data
import yfinance as yf

# Class(s) for Data analysis and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Class for  Connecting to Interactive Brokers
from ib_insync import *
import nest_asyncio

# Class for portfolio optimization
import riskfolio as rf

# Class(s) for supportive functions
from datetime import datetime as dt
import math
import random
import copy
from time import sleep
import warnings
from waiting import wait, TimeoutExpired

In [ ]:
pd.options.display.float_format = "{:.4%}".format
warnings.filterwarnings("ignore")

In [ ]:
#List of stocks currently listed on the Nasdaq
nasdaq_stocks = pd.read_html("http://en.wikipedia.org/wiki/Nasdaq-100#Components", match="Company")[0]
nasdaq_stocks.head()

In [ ]:
nasdaq_ticker_list = nasdaq_stocks.Ticker.tolist()

In [ ]:
sp_stocks = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp_stocks.head()

In [ ]:
sp_ticker_list = sp_stocks.Symbol.tolist()

In [ ]:
# Changing the ticker format to match the format in Yahoo finance for the following BRK.B and BF.B
sp_ticker_list[sp_ticker_list.index('BRK.B')] = 'BRK-B'
sp_ticker_list[sp_ticker_list.index('BF.B')] = 'BF-B'

In [ ]:
# Combining the two lists and removing duplicates
long_ticker = list(set(sp_ticker_list + nasdaq_ticker_list))

In [ ]:
long_df = pd.DataFrame()
failed_tickers = []
for ticker in long_ticker:
    try:
        long_df[ticker] = yf.download(ticker, period = '3y', interval = '1mo')['Close']
        # Calculate technical indicators
        # Using Simple Moving Average to determine Stocks that have upward momentum
        long_df[f'{ticker}_SMA5'] = long_df[ticker].rolling(window=5).mean()
        long_df[f'{ticker}_SMA15'] = long_df[ticker].rolling(window=15).mean()
        long_df[f'{ticker}_prev_SMA5'] = long_df[f'{ticker}_SMA5'].shift(1)
        long_df[f'{ticker}_prev_SMA15'] = long_df[f'{ticker}_SMA15'].shift(1)

    except Exception as e:
        failed_tickers.append(ticker)

In [ ]:
# Check if UBER_SMA5 > UBER_SMA15 for the specific date
condition = (long_df['UBER_SMA5'] > long_df['UBER_SMA15']) & (long_df['UBER_prev_SMA5'] >= long_df['UBER_prev_SMA15'])
filtered_data = long_df[condition]

# Get the row for the last date if it exists in the filtered data
if long_df.index[-1] in filtered_data.index:
    result = filtered_data.loc[long_df.index[-1]]
    print(condition[-1])
    # print(f"Data for {long_df.index[-1]} where UBER_SMA5 > UBER_SMA15:")
    # print(result)
else:
    print("No data for 2025-08-01 where UBER_SMA5 > UBER_SMA15")
    # Show the closest available date
    available_dates = filtered_data.index
    if len(available_dates) > 0:
        print(f"Available dates where condition is met: {available_dates[-5:]}")  # Show last 5 dates

In [ ]:
bullish_list=[]
bearish_list=[]
error_list=[]
for ticker in long_ticker:
    try:
        condition = (long_df[f'{ticker}_SMA5'] > long_df[f'{ticker}_SMA15']) & (long_df[f'{ticker}_prev_SMA5'] >= long_df[f'{ticker}_prev_SMA15'])
        if condition.iloc[-1] == True:
            bullish_list.append(ticker)
        else:
            bearish_list.append(ticker)
    except Exception as e:
        error_list.append((ticker, str(e)))

bullish_list

In [ ]:
bearish_list

In [ ]:
print(len(bullish_list))
print(len(bearish_list))

In [ ]:
# Calculating the performance KPIs

# Calculating the Compounded Annual Growth Rate
def CAGR(DF):
    df = DF.copy()
    df['cum_return'] = (1 + df['mon_return']).cumprod()
    n = len(df)/12 # the denominator is the number of trading periods in a year. In this case, 12 months
    CAGR = (df['cum_return'].tolist()[-1]) ** (1/n)-1
    return CAGR

# Calculating the Volatility
def volatility(DF):
    df = DF.copy()
    vol  = df['mon_return'].std() * np.sqrt(12) # The number in the sqrt is the number of r trading periods in a year
    return vol

# Calculating the Sharpe Ratio
def sharpe(DF, rf):
    df = DF.copy()
    sharpe = (CAGR(df) - .03) / volatility(df)
    return sharpe

# Calulating the maximum drawdown
def max_dd(DF):
    df = DF.copy()
    df['cum_return'] = (1 + df['mon_return']).cumprod()
    df['cum_roll_max'] = df['cum_return'].cummax()
    df['drawdown'] = df['cum_roll_max'] - df['cum_return']
    df['drawdown_pct'] = df['drawdown'] / df['cum_roll_max']
    max_dd = df['drawdown_pct'].max()
    return max_dd

In [ ]:
def pflio(DF,m,x):
    """Returns cumulative portfolio return
    DF = dataframe with monthly return info for all stocks
    m = number of stocks to keep in the portfolio
    x = number of underperforming stocks to be removed from portfolio monthly"""
    df = DF.copy()
    portfolio = []
    monthly_ret = [0]
    for i in range(len(df)):
        if len(portfolio) > 0:
            monthly_ret.append(df[portfolio].iloc[i,:].mean())
            bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=True)[:x].index.values.tolist()
            portfolio = [t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_picks = df.iloc[i,:].sort_values(ascending=False)[:fill].index.values.tolist()
        portfolio = portfolio + new_picks
        print('\n list of stocks to go long: \n', portfolio)
    return portfolio

In [ ]:
# Creating an empty dictionary which will be filled with Open, High, Low, Close, Volume dataframe for each ticker
ohlcv_data = {} 
for ticker in bullish_list: 
    ohlcv_data[ticker] = yf.download(ticker, period = '5y', interval = '1mo')
    ohlcv_data[ticker].dropna(inplace = True, how = 'all')
bullish_tickers = ohlcv_data.keys()

In [ ]:
ohlcv_dict = copy.deepcopy(ohlcv_data)
bullish_return_df = pd.DataFrame()
for ticker in bullish_tickers:
    print('calculating monthly return for',ticker)
    ohlcv_dict[ticker]['mon_return'] = ohlcv_dict[ticker]['Close'].pct_change()
    bullish_return_df[ticker] = ohlcv_dict[ticker]['mon_return']
bullish_return_df.dropna(how='all',axis=0, inplace=True)

In [ ]:
long_list = pflio(bullish_return_df,20,10)

In [ ]:
# Sleeping 1 minute since over 500 calls was made to yfinance
sleep(60)
port_returns = (
    yf.download(
        long_list,
        period  = '1mo' 
    )["Close"]
    .pct_change()
    .dropna()
)

In [ ]:
# These factors is a list of indicies that aims to track the performance
# of a particular startegy applied to the U.S. equity market

factors = ["MTUM", "QUAL", "VLUE", "SIZE", "USMV"]

In [ ]:
factor_returns = (
    yf.download(
        factors, 
        period  = '1mo'
    )["Close"]
    .pct_change()
    .dropna()
)

In [ ]:
port = rf.Portfolio(returns = port_returns)

port.assets_stats(method_mu = "hist", 
                  method_cov = "ledoit"
                  )

port.lowerret = 0.00056488 * 1.5

loadings = rf.loadings_matrix(
    X = factor_returns,
    Y = port_returns, 
    feature_selection = "PCR",
    n_components = 0.95
)

In [ ]:
loadings.style.format("{:.4f}").background_gradient(cmap='RdYlGn')

In [ ]:
port.factors = factor_returns

port.factors_stats(
    method_mu = "hist",
    method_cov = "ledoit",
)

In [ ]:
w = port.optimization(
    model = "FM",
    rm = "MV",
    obj = "Sharpe",
    hist = True
)

In [ ]:
w.reset_index(inplace = True)
w.rename(columns={'index':'ticker', 'weights':'weights'}, inplace = True)
w.sort_values('weights')

In [ ]:
w[w['weights'] < .01] = np.nan
w.dropna(inplace = True)
w


In [ ]:
# Sorting the weights in descending order
w.sort_values('weights', ascending=False)

In [ ]:
long_dict = {} # empty dictionary which will be filled with ohlcv dataframe for each ticker
op_ticker_list = w['ticker'].to_list()
for long_op_ticker in op_ticker_list:
    long_dict[long_op_ticker] = yf.download(long_op_ticker, period = '1mo')
    long_dict[long_op_ticker].dropna(inplace = True, how = 'all')
tickers = long_dict.keys()

In [ ]:
nest_asyncio.apply()

# Connect to IB Gateway
# ib = IB()
# ib.connect('127.0.0.1', 4002, clientId=random.randint(1, 99))

# Connect to IB TWS
ib = IB()
ib.connect('127.0.0.1', 7497, clientId = random.randint(1,99))

In [ ]:
def buy_stock(long_ticker: str, buy_diff: float = None):
        
        # Checking the account balance
        acct_bal = float(ib.accountSummary()[9].value)
        
        stock = Stock(
            symbol = long_ticker, 
            exchange = 'SMART', 
            currency = 'USD'
        )
        
        purchase_amount = ((float(w[w['ticker'] == f'{long_ticker}']['weights']) * acct_bal)) / (long_dict[long_ticker]['Close'].iloc[-1])

        if buy_diff is None:
            action = Order(
                action = 'BUY', 
                totalQuantity = round(purchase_amount.item()), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )
        
        else:
            action = Order(
                action = 'BUY', 
                totalQuantity = math.ceil(buy_diff), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )  
        
        order = ib.placeOrder(stock, action)

In [ ]:
def sell_stock(ticker: str, sell_diff: float = None):
        stock = Stock(
            symbol = ticker, 
            exchange = 'SMART', 
            currency = 'USD'
        )
        
        for i in range (len(ib.positions())):
            if ib.positions()[i].contract.symbol == ticker:
                sell_amount = ib.positions()[i].position

        if sell_diff is None:
            action = Order(
                action = 'SELL', 
                totalQuantity = sell_amount, 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            )
        else:
            action = Order(
                action = 'SELL', 
                totalQuantity = math.ceil(sell_diff), 
                orderType = 'MKT',  
                tif = 'GTC', 
                outsideRth = True
            ) 
        
        order = ib.placeOrder(stock, action)

In [ ]:
current_holdings = []

for i in range(len(ib.positions())):
    current_holdings.append(ib.positions()[i].contract.symbol)

In [ ]:
current_holdings = []

for i in range(len(ib.positions())):
    current_holdings.append(ib.positions()[i].contract.symbol)

for holdings in current_holdings:
    if holdings not in long_list: 
        try:
            sell_stock(holdings)
        except TimeoutExpired as timeout:
            print(f'Sale of {holdings} {timeout} ')
    
    elif holdings in long_list:
        for i in range(len(ib.positions())):
            if ib.positions()[i].contract.symbol == holdings:
                current_holding_port_percentage = ib.positions()[i].position * ib.positions()[i].avgCost / float(ib.accountSummary()[24].value)
                if not w[w['ticker'] == f'{holdings}'].empty:
                    new_holding_port_percentage = w[w['ticker'] == f'{holdings}']['weights'].iloc[0]
                    
                    if current_holding_port_percentage > new_holding_port_percentage:
                        sell_diff = current_holding_port_percentage - new_holding_port_percentage
                        try:
                            sell_stock(ticker = holdings, sell_diff = sell_diff)
                        except TimeoutExpired as timeout:
                            print(f'Sale of {holdings} has been after {timeout}')
                    
                    elif current_holding_port_percentage < new_holding_port_percentage:
                        buy_diff = new_holding_port_percentage - current_holding_port_percentage
                        try:
                            buy_stock(long_ticker = holdings, buy_diff = buy_diff)
                        except ValueError as e:
                            print(f'Error buying {holdings}: {e}')
                else:
                    sell_stock(holdings)
                    print(f'{holdings} not found in the weights DataFrame, Therefore {holdings} was sold')
    

In [ ]:
for long_op_ticker in op_ticker_list:
    if long_op_ticker not in current_holdings:
        try:
            buy_stock(long_op_ticker)
            ib.sleep(15)
        except ValueError as e:
            print(f'Error buying stock {long_op_ticker}: {e}')

In [ ]:
# Get current positions
positions = ib.positions()

# Print the positions
for position in positions:
    print(f"Symbol: {position.contract.symbol}, Quantity: {position.position}")


In [ ]:
# Loop through your positions and creating trailing stop orders
for position in positions:
    contract = position.contract
    quantity = position.position
    
    if quantity > 0:  # Only create trailing stop orders for long positions
        # Define the trailing stop order
        trailing_stop_order = Order(
            action = 'SELL',  # Sell to exit the position
            orderType = 'TRAIL',
            totalQuantity = abs(quantity),
            trailingPercent = 10.0,  # Adjust the trailing percentage as needed
            tif = 'GTC'
        )

        # Submit the order
        trade = ib.placeOrder(contract, trailing_stop_order)
        print(f"Trailing stop order placed for {contract.symbol}, to sell a Quantity of {quantity}")

In [ ]:
# Fetch open orders
open_orders = ib.openOrders()
for open_order in open_orders:
    print(f"Open Orders: {open_orders}")

ib.disconnect()